In [1]:
%load_ext autoreload
%autoreload 2
from glob import glob
import numpy as np
from utils import compare_product_color,remove_existing,multi_process_product
from sklearn.utils import shuffle
from tqdm import tqdm
from PIL import Image
import random
import uuid
import os
import pandas as pd
import faiss
import multiprocessing as mp
import gc
from shutil import copyfile

In [2]:
_META_DATA_PATH = '../../data/meta_data'
_RESOURCE_PATH = '../../resource'

In [3]:
i = 28

In [4]:
batch_prod = 100
list_prod_id = glob(os.path.join(_META_DATA_PATH,'*'))
list_prod_id = sorted(list_prod_id)
# list_prod_id = list_prod_id[i*batch_prod:(i+1)*(batch_prod)]
file_name_tracking = str(i*batch_prod)+'_'+str(len(list_prod_id)) + '.csv'
list_prod_id = list_prod_id[i*batch_prod:]

list_tracking = []
tuple_result_multiprocess = None

In [5]:
file_name_tracking

'2800_2851'

In [6]:
for prod_id in tqdm(list_prod_id):
    list_ids = glob(os.path.join(prod_id,'*.jpg'))
    list_ids = sorted(list_ids)
    try:
        if len(list_ids) > 1:
            tuple_result_multiprocess = multi_process_product(list_ids)

            list_pair = []

            for item1 in list_ids:
                sublist_pair = []
                for item2 in list_ids:
                    pair = [item1,item2]
                    sublist_pair.append(pair)
                list_pair.append(sublist_pair)

            pair_array = np.array(list_pair)
            list_pair_triu = pair_array[np.triu_indices(pair_array.shape[0])]

            list_pair_rm_dup = []
            for item in list_pair_triu.tolist():
                if item[0] != item[1]:
                    list_pair_rm_dup.append(item)

            list_cmp = []

            for p in list_pair_rm_dup:
                sublist_cmp_group = []
    #             print(p)
                ## cal list centroids

                cmp = compare_product_color(p,tuple_result_multiprocess)
                if cmp:
                    ## seperate
                    ## candidate for group key
                    ## key name sorted
                    if list_cmp:
                        if not p[0] in list(set(sum(list_cmp, []))):
                            list_cmp.append([p[0]])
                        if not p[1] in list(set(sum(list_cmp, []))):
                            list_cmp.append([p[1]])
                    else:
                        list_cmp.append([p[0]])
                        list_cmp.append([p[1]])
                else:
                    sublist = []
                    list_cmp.append(sublist)

                    for group_exist in list_cmp:
                        if p[0] in group_exist and p[1] not in group_exist:
                            group_exist.append(p[1])
                        elif p[1] in group_exist and p[0] not in group_exist:
                            group_exist.append(p[0])
                        elif not p[0] in list(set(sum(list_cmp, []))) or not p[1] in list(set(sum(list_cmp, []))):
                            group_exist.append(p[0])
                            group_exist.append(p[1])

            list_cmp_sort = [sorted(item) for item in list_cmp]
            list_cmp_rm_dup = remove_existing(list_cmp_sort)
    #         list_cmp_short = [[subitem.split('/')[-1] for subitem in item] for item in list_cmp_rm_dup]

            for idx, group in enumerate(list_cmp_rm_dup):
                for item in group:
                    dict_tracking = {}
                    dict_tracking['product'] = prod_id.split('/')[-1]
                    dict_tracking['path'] = item
                    dict_tracking['group'] = idx
                    list_tracking.append(dict_tracking)

        else:           
            dict_tracking = {}
            dict_tracking['product'] = prod_id.split('/')[-1]
            dict_tracking['path'] = list_ids[0]
            dict_tracking['group'] = 0
            list_tracking.append(dict_tracking)
            
        
    except Exception as e:
        print('Fail at {0} caused {1}'.format(prod_id,str(e)))

100%|██████████| 51/51 [03:10<00:00,  3.74s/it]


In [7]:
df_tracking = pd.DataFrame(list_tracking)

df_tracking.to_csv(os.path.join(_RESOURCE_PATH,file_name_tracking),header=True,index=False)